In [1]:
import swissgrid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holidays
from sklearn.linear_model import LinearRegression

In [2]:
ts = swissgrid.grid_data.iloc[:,0].fillna(method='bfill')
SAMPS_DAY = 24*4

In [3]:
weather = pd.read_csv('data/weather_data_filtered.csv')
weather['timestamp'] = pd.to_datetime(weather['utc_timestamp'])
weather = weather[['timestamp', 'CH_temperature']].set_index('timestamp').sort_index()
weather = weather.resample('1 H').first()
weather = weather.tz_localize(None)
weather = weather.resample('15 T').interpolate('linear')

In [4]:
# align the two sequences
ts = ts[:weather.index[-1]]
weather = weather[ts.index[0]:]

In [5]:
days = ts.index
# special days
def get_holy(prov):
    h = holidays.Switzerland(prov=prov)
    return [day in h for day in days]
holidays_cantons = pd.DataFrame(data = {prov:get_holy(prov) for prov in holidays.Switzerland.PROVINCES},
             index = days)
#add sundays (hurts MSE)
# holidays_cantons = (holidays_cantons.T | (holidays_cantons.index.dayofweek == 6)).T

cantons = pd.read_csv('data/cantons.csv').set_index('Code')
#magic trick
cantons['Population'] = cantons['Population'].str.extract('([^\[]*)').iloc[:,0].str.split(',').str.join('').astype(np.intp)

holiday_pop = holidays_cantons*cantons['Population']

### autoregressive part

The consumption $L_{hd}$ today at day $d$ and hour $h$ depends on $L_{h-1d}$, $L_{hd-1}$, $L_{hd-7}$

In [116]:
def build_ts_regressor(ts, fourier_order=4):
    #interact annual pattern like (6)
    week_lag_idx = np.arange(ts.size-SAMPS_DAY*7)
    t = ((
        ts.index[SAMPS_DAY*7:ts.size] - pd.datetime(year=ts.index[0].year, month=1, day=1)
    ).total_seconds()//(60*24*60/SAMPS_DAY)).astype(np.intp)
    samps_year = SAMPS_DAY*7*52
    sins =  np.array([np.sin(2*q*np.pi*(t/samps_year)) for q in range(1, fourier_order+1)]).T
    cosins =np.array([np.cos(2*q*np.pi*(t/samps_year)) for q in range(1, fourier_order+1)]).T
    week_lagged_reg = ts.values[week_lag_idx,None]*np.hstack([np.ones((week_lag_idx.size,1)), sins, cosins])
    
    #interact daily indicators with daily lag (5)
    week_day = ts.index[SAMPS_DAY*7:ts.size].dayofweek
    I = np.eye(7)
    week_day_indicators = np.array([I[i] for i in week_day])
    day_lag_idx = np.arange(SAMPS_DAY*6, ts.size-SAMPS_DAY)
    day_lagged_reg = ts.values[day_lag_idx,None]*week_day_indicators
    
    # hourly lag regressor (7)
    hour_lag_idx = np.arange(SAMPS_DAY*7 - 1, ts.size-1)
    hour_lagged_reg = ts.values[hour_lag_idx][:,None]
    
    #temperature
    weather_reg = weather.values[SAMPS_DAY*7:ts.size]
    
    #special days
    holiday_reg = holiday_pop.sum(axis=1).values[SAMPS_DAY*7:ts.size, None]
    holiday_lagged_reg = holiday_pop.sum(axis=1).values[:ts.size - SAMPS_DAY*7,None]
    
    return np.hstack(
        [
            week_lagged_reg,
            day_lagged_reg,
            hour_lagged_reg,
            weather_reg,
            holiday_reg,
            holiday_lagged_reg
        ]
    )

In [117]:
reg = build_ts_regressor(ts)
lr = LinearRegression().fit(reg, ts.values[7*SAMPS_DAY:])

In [118]:
MAPE_triv = np.mean(np.abs((ts.values[7*SAMPS_DAY-1:-1] - ts.values[7*SAMPS_DAY:]))/ts.values[7*SAMPS_DAY:])
MAPE_triv

0.013428251107830377

In [119]:
MAPE = np.mean(np.abs((lr.predict(reg) - ts.values[7*SAMPS_DAY:]))/ts.values[7*SAMPS_DAY:])
MAPE

0.01285939996303295

### Moving Average part

We can now add a moving average component to the mix and see if our forecasting imporves. The moving average takes into account previous hour errors and previous week error.
The estimation is done by applying the ordinary linear model, and treating residuals as estimated innovations, and iterating the least square estimation, and innovation estimations.

In [166]:
reg = build_ts_regressor(ts)
eps = np.zeros_like(ts.values[7*SAMPS_DAY:])
# we drop an additional leading week so the least squares estimation is done
# with all the estimated epsilons, previous week before included.
for c in range(100):
    eps_reg = np.hstack([eps[:-7*SAMPS_DAY,None], eps[6*SAMPS_DAY:-SAMPS_DAY,None]])
    lr = LinearRegression().fit(np.hstack([reg[7*SAMPS_DAY:],eps_reg]), ts.values[14*SAMPS_DAY:])
    eps = ts.values[7*SAMPS_DAY:] - lr.predict(np.hstack([reg, np.vstack([np.zeros([7*SAMPS_DAY,eps_reg.shape[1]]), eps_reg])]))
    print(c, np.mean(np.abs(eps[7*SAMPS_DAY:]/ts.values[14*SAMPS_DAY:])))

0 0.012861128884652464
1 0.006131352722112417
2 0.01218793396426801
3 0.007354190132208505
4 0.011657035501870517
5 0.008094055598941696
6 0.011263047348580007
7 0.008525959288165845
8 0.011013724173623438
9 0.008783827920633988
10 0.01085655438386978
11 0.008946072964102627
12 0.010737718725811371
13 0.009080013801032218
14 0.010633772619129244
15 0.009208882152856295
16 0.010532886637025628
17 0.009327931791110257
18 0.010439736953300282
19 0.009431054572513869
20 0.010358303805496866
21 0.009513243363826532
22 0.010290910840467194
23 0.009577446624755933
24 0.010239559604734116
25 0.009628432047370796
26 0.010195690198762332
27 0.009670458233369244
28 0.010158014492929506
29 0.009706256685063747
30 0.010126506388795711
31 0.009737940699068293
32 0.01009789576660453
33 0.009765492065093329
34 0.010072756187669061
35 0.009789924349569533
36 0.010051486213568802
37 0.009810504018279622
38 0.010033147321825501
39 0.009827946100736154
40 0.010017696292482716
41 0.009843093893025293
42 0.

## Multiple Equations

By changing the single regression into 96 individual parameter sets, one for each daily time, we are back to the multiple equations setting described in Clements, Hurn and Li.

In [195]:
reg = build_ts_regressor(ts)
eps = np.zeros_like(ts.values[7*SAMPS_DAY:])
regressions = [LinearRegression() for i in range(SAMPS_DAY)]

for c in range(1000):
    eps_reg = np.hstack([eps[:-7*SAMPS_DAY,None], eps[6*SAMPS_DAY:-SAMPS_DAY,None]])
    for daily_time in range(SAMPS_DAY):
        regressions[daily_time].fit(
            np.hstack(
                [reg[7*SAMPS_DAY+daily_time::SAMPS_DAY],eps_reg[daily_time::SAMPS_DAY]]
            ),
            ts.values[daily_time + 14*SAMPS_DAY::SAMPS_DAY]
        )
        eps[daily_time::SAMPS_DAY] = ts.values[daily_time + 7*SAMPS_DAY::SAMPS_DAY]\
            - regressions[daily_time].predict(
                np.hstack([
                        reg[daily_time::SAMPS_DAY],
                        np.vstack([np.zeros([7*SAMPS_DAY,eps_reg.shape[1]]), eps_reg])[daily_time::SAMPS_DAY]
                ])
            )
    if c % 100 == 0:
        print(c, np.mean(np.abs(eps[7*SAMPS_DAY:]/ts.values[14*SAMPS_DAY:])))

0 0.004430604714915216
100 0.004310552310005237
200 0.0043105523100036534
300 0.0043105523100036135
400 0.004310552310003565
500 0.004310552310003626
600 0.004310552310003822
700 0.004310552310003665
800 0.004310552310003729
900 0.004310552310003775


In [188]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels import api as sm

In [222]:
# for c_i in [10,11,13,14,15,16]:
#     plt.plot([regressions[i].coef_[12] - regressions[i].coef_[c_i] for i in range(SAMPS_DAY)]);
# plt.legend(['monday','tuesday','thursday','friday','saturday','sunday'])

# this plot is not similar at all to that of the paper


In [223]:
#acf should be more or less flat?

# plot_acf(eps, lags=np.arange(0,22));